<a href="https://colab.research.google.com/github/mirpouya/Transformer_EDU/blob/main/einsum_and_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Think about the situation that you want to merge d dimsnsion of a 4D matrix

In [1]:
import torch
from torch import nn

In [2]:
rand_tensor = torch.rand((2, 5))
rand_tensor

tensor([[0.8782, 0.9865, 0.0878, 0.2703, 0.7359],
        [0.1179, 0.7844, 0.8179, 0.5407, 0.9978]])

In [3]:
rand_tensor = torch.rand((3,4,5,6))
rand_tensor

tensor([[[[0.1378, 0.0724, 0.4583, 0.5448, 0.3950, 0.5617],
          [0.9874, 0.9202, 0.7649, 0.8667, 0.7688, 0.9397],
          [0.7262, 0.6742, 0.6526, 0.2905, 0.5091, 0.9446],
          [0.4610, 0.8897, 0.4249, 0.2802, 0.1738, 0.7169],
          [0.4922, 0.9954, 0.8705, 0.2483, 0.8635, 0.3960]],

         [[0.4481, 0.8084, 0.0134, 0.0274, 0.0634, 0.8233],
          [0.7449, 0.4815, 0.3212, 0.9034, 0.5738, 0.1587],
          [0.4295, 0.9350, 0.0716, 0.9254, 0.6426, 0.9469],
          [0.1661, 0.7564, 0.0913, 0.7325, 0.6468, 0.0161],
          [0.1581, 0.6332, 0.1155, 0.3457, 0.6860, 0.8658]],

         [[0.5001, 0.7243, 0.4377, 0.8629, 0.0585, 0.5619],
          [0.7507, 0.5979, 0.7335, 0.3665, 0.3047, 0.0133],
          [0.3859, 0.2861, 0.9227, 0.9089, 0.9121, 0.8607],
          [0.6747, 0.2502, 0.4391, 0.7914, 0.9292, 0.6879],
          [0.5172, 0.8294, 0.1045, 0.1152, 0.6105, 0.2560]],

         [[0.7807, 0.1589, 0.7507, 0.4653, 0.9658, 0.2052],
          [0.2367, 0.7955, 0.4479,

In [4]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00


In [5]:
from einops import rearrange, reduce, repeat

In [6]:
rand_tensor.size()

torch.Size([3, 4, 5, 6])

In [7]:
rand_tensor = rearrange(rand_tensor, "b c h w -> (b w) c h")

# b = 3, c = 4, h = 5, w = 6
# to (3 * 6, 4, 5)

In [8]:
rand_tensor.size()

torch.Size([18, 4, 5])

### <b> einsum </b>

In [9]:
a = torch.rand((10, 10, 30))
a.size()

torch.Size([10, 10, 30])

In [10]:
a = torch.rand(10, 20, 30)
a.size()

torch.Size([10, 20, 30])

In [11]:
a = torch.randn(10, 20, 30)  # b = 10, i = 20, k = 30
b = torch.randn(10, 50, 30)  # b = 10, i = 50, k = 30

In [12]:
y = torch.einsum("b i k, b j k -> b i j", a, b)

In [13]:
y.size()

torch.Size([10, 20, 50])

In [14]:
import numpy as np

A = np.array([0, 1, 2])

B = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])

In [15]:
np.einsum('i,ij->i', A, B)

array([ 0, 22, 76])

# <b> Transformer's blocks implementation </b>

## <b> Scaled dot product self-attention </b>

In the article, after embedding and positional embedding, embeddings are multiplied by weight matrices  <b> W_Q </b>,  <b> W_K </b>,  <b> W_V </b>.

the input <b> X </b> to the attention block is of shape: (batch, sequence_len, embedding_dim)

The matrix multiplication happens in the <b> embedding_dim </b> dimension, regardless of batch_size and sequence_len

In [16]:
# linear projection before attention block
embed_dim = 512
input = torch.randn(10, 12, 512)   # for example

qkv_weights = nn.Linear(embed_dim, embed_dim * 3, bias = False)  # we concat q, k, v into one matrix, then multiply by qkv_weights

qkv = qkv_weights(input)

q, k, v = tuple(rearrange(qkv, "b t (d k) -> k b t d", k = 3))

In [17]:
q.size(), k.size(), v.size()

(torch.Size([10, 12, 512]),
 torch.Size([10, 12, 512]),
 torch.Size([10, 12, 512]))

<b> Step 2 </b>

calculate scaled dot product, apply mask, and finally compute softmax in d last dimension.

In [18]:
# scaled dot product
scale_factor = 512 / 8   # embed_dim / num_heads
scaled_dot_product = torch.einsum("b i d, b j d -> b i j",q, k) * scale_factor
# resulting shape: (batch_size, embed_dim, embed_dim)

# masking if needed (decoder)
mask = None
if mask is not None:
  assert mask.shape == scaled_dot_product.shape[1:]
  scaled_dot_product = scaled_dot_product.masked_fill(mask, -np.inf)

attention = torch.softmax(scaled_dot_product, dim = -1)

# multiply attention scores with V
print(f"attention size: {attention.size()}")
print(f"value size: {v.size()}")

attention_final = torch.einsum("b i d, b d j -> b i j", attention, v)

attention size: torch.Size([10, 12, 12])
value size: torch.Size([10, 12, 512])


## <b> Implementation of Scaled dot Prodcut Self-Attention </b>

In [19]:
import torch
from torch import nn

import numpy
from einops import rearrange

In [20]:
class SelfAttention(nn.Module):

  def __init__(self, embed_dim):
    """
    Args:
        embed_dim: embedding dimension, with 512 as default
        the last dimension size that is provided in forward(x), where x is a 3D tensor
    """
    super().__init__()

    # wieght matrices for query, key, and value
    self.qkv_weights = nn.Linear(embed_dim, embed_dim * 3, bias = False)
    self.scale_factor = embed_dim ** (-0.5)

  # forward method
  def forward(self, x, mask = None):
    assert x.dim() == 3  # x must be a 3D tensor (batch_size, sequence_len, embed_dim)

    # step 1
    qkv = self.qkv_weights(x)

    # step 2: decomposing to q, k, v
    # rearranging to [3, batch_size, sequence_len, embed_dim]
    q, k , v = tuple(rearrange(qkv, "b t (d k) -> k b t d", k = 3))

    # scaled_dot_product
    scaled_dot_product = torch.einsum("b i d, b j d -> b i j", q, k) * self.scale_factor

    # mask attention
    if mask is not None:
      assert mask.shape == x.shape[1:]
      scaled_dot_product = scaled_dot_product.masked_fill(mask, -np.inf)

    attention = torch.softmax(scaled_dot_product, dim = -1)
    # you have to multiply V in the dimension you apply the softmax. Be careful of that.
    attention_final = torch.einsum("b i d, b d k -> b i k")

    return attention_final

## <b> Implementing Multi-Head_Self-Attention </b>

in a single head case, we project the embedded and positioonal encoded input to weight matrix of size (embed_dim, embed_dim * 3)

in multi-head case, we project the inuput matrix to weigh matrix of size (embed_dim, head_dim * n_heads * 3), which is the same, but in the `rearrange()` step it is easy to separate heads.

In [21]:
head_dim = 512 // 8
n_heads = 8
x = torch.randn(10, 12, 512)

qkv_weights = nn.Linear(embed_dim, head_dim * n_heads * 3, bias=False)
qkv = qkv_weights(x)

In [22]:
# decompose qkv to q, k, v
q, k, v = tuple(rearrange(qkv, "b s (h n k) -> k b n s h", k = 3, n = n_heads))

In [23]:
qkv.size()

torch.Size([10, 12, 1536])

In [24]:
q.size(), k.size(), v.size()

(torch.Size([10, 8, 12, 64]),
 torch.Size([10, 8, 12, 64]),
 torch.Size([10, 8, 12, 64]))

the next step is to calculate `scaled-dot-product`, apply the mask, and finally compute the `softmax` in `dim_head`

In [25]:
# matrix multiplication of q and v in heads
# q, k -> (batch_size, n_heads, seq_len, dim_head)
scaled_dot_product = torch.einsum("b n s d, b n t d -> b n s t", q, k) * scale_factor   # (batch_size, n_heads, seq_len, tokens) "seq_len = tokens"

if mask is not None:
  # check mask shape
  assert mask.shape == scaled_dot_product.shape[2:]
  scaled_dot_product = scaled_dot_product.masked_fill(mask, -np.inf)

attention = torch.softmax(scaled_dot_product, dim = -1)

now that the attention is computed, we must multiply the attention score of each head with the corresponding value of each head

In [26]:
# attention shape : (batch_size, n_heads, sentence_words, sentence_words)
# value shape : (batch_size, n_heads, sentence_words, head_dim)

out = torch.einsum("b n i j, b n j d -> b n i d", attention, v)

it's time to merge heads into one matrix and multiply it by W_O

In [27]:
# concatenating heads
out = rearrange(out, "b n i d -> b i (n d)")

MultiHead(Q, K, V) = Concat(head1, ..., head8) <br>
head_i = Attention(QW_Q,i, KW_K,i, VW_V,i)

In [28]:
# apply linear transformation W_O
W_O = nn.Linear(embed_dim, embed_dim, bias = False)

final_output = W_O(out)

## <b> Implementation of Multi-Head-Attention </b>

In [29]:
import torch
from torch import nn
from einops import rearrange
import numpy as np

class MultiHeadAttention(nn.Module):
  def __init__(self, embed_dim, n_heads):
    super().__init__()

    self.n_heads = n_heads
    self.dim_head = embed_dim // self.n_heads
    self.embed_dim = self.dim_head * n_heads
    self.scale_factor = self.dim_head ** (-0.5)

    # weight matrices
    self.qkv_weights = nn.Linear(self.embed_dim, n_heads * self.dim_head * 3, bias = False)
    self.W_O = nn.Linear(self.embed_dim, self.embed_dim, bias = False)

  # forward method
  def forward(self, x, mask = None):
    # check x has all 3 dimensions -> (batch_size, sentence_length, embedding_dim)
    assert x.shape == 3

    # step 1: compute query, key, value
    qkv = self.qdv_weights(x)   # (batch_size, sentence_length, dim_head * n_heads * 3)

    # step 2: decompose to q, k, v
    # resulting shape befor tuple():
    # [3, n_heads, batch_size, sentence_len, head_dim]
    q, k, v = tuple(rearrange(qkv, "b s (d n k) -> k b n s d", k = 3, h = self.n_heads))

    # step 3: compute scaled_dot_product
    scaled_dot_product = torch.einsum("b n s d, h n t d -> b n s t", q, v) * self.scale_factor

    # mask if needed
    if mask is not None:
      assert mask.shape == scaled_dot_product[2:]
      scaled_dot_product = scaled_dot_product.masked_fill(mask, -np.inf)

    attention = torch.softmax(scaled_dot_product, dim = -1)

    # step 4: calculate output
    out = torch.einsum("b n s j, b n j d -> b n s d", attention, v)

    # step 5: merge heads
    out = rearrange(out, "b n i d -> b s (n d)")

    # step 6: apply W_O
    output = self.W_O(out)

    return output

## <b> Transformer Encoder Block </b>

In [31]:
class TransformerBlock(nn.Module):

  def __init__(self, dim, heads = 8, dim_head = None, dim_linear_block = 1024, dropout = 0.1):
    """
    dim: tocken's vector lenght
    heads: number of heads
    dim_head: if None, dim / heads is used
    dim_linear_block: linear projection dimension following MultiHeadAttention
    dropout: probability of dropping out
    """
    super().__init__()
    self.mha = MultiHeadAttention(embed_dim = 512, n_heads = heads)
    self.drop = nn.Dropout(dropout)
    self.norm1 = nn.LayerNorm(dim)
    self.norm2 = nn.LayerNorm(dim)

    self.linear = nn.Sequential(
        nn.Linear(dim, dim_linear_block),
        nn.ReLU(),
        nn.Linear(dim_linear_block, dim),
        nn.Dropout(dropout)
    )

  # forward method
  def forward(self, x, mask = None):
    y = self.norm1(self.drop(self.mha(x, mask) + x))
    return self.norm2(self.linear(y) + y)